In [1]:
import pandas as pd

In [10]:
business = pd.read_csv('../data/YelpDatasetKaggle/yelp_academic_dataset_business.csv')

C:\Users\ineso\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (56) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
import re
business['restaurant']=business['categories'].str.contains('Restaurants',flags=re.IGNORECASE)

In [ ]:
business_restaurant=business[business['restaurant']==True]

In [ ]:
business_restaurant.reset_index(drop=True).head()

In [13]:
business_restaurant.to_pickle('../data/all-restaurants/restaurants.pickle')

In [3]:
business_restaurant = pd.read_pickle('../data/all-restaurants/restaurants.pickle')

In [ ]:
business_restaurant.head()

In [12]:
business_restaurant.shape

(50743, 60)

In [ ]:
reviews = pd.read_csv('../data/YelpDatasetKaggle/yelp_academic_dataset_review.csv')
review_restaurant = reviews.loc[reviews['business_id'].isin(business_restaurant['business_id'].unique())]

In [ ]:
review_restaurant.reset_index(drop=True).head()

In [2]:
review_restaurant = pd.read_pickle('../data/all-restaurants/restaurant_review.pickle')

In [3]:
user_rev_count = review_restaurant.user_id.value_counts()

In [4]:
users2remove = []
for user, count in user_rev_count.items():
    if count < 3:
        users2remove.append(user)

In [5]:
index_names = review_restaurant[review_restaurant['user_id'].isin(users2remove)].index
review_restaurant.drop(index_names, inplace=True)

In [9]:
review_restaurant.to_pickle('../data/all-restaurants/restaurant_review.pickle')

In [6]:
review_restaurant.shape

(4197120, 9)

In [6]:
user = pd.read_csv('../data/YelpDatasetKaggle/yelp_academic_dataset_user.csv')

In [7]:
user_restaurant=user.loc[user['user_id'].isin(review_restaurant['user_id'].unique())]

In [8]:
user_restaurant.to_pickle('../data/all-restaurants/restaurant_users.pickle')

In [ ]:
user_restaurant = pd.read_pickle('../data/all-restaurants/restaurant_users.pickle')

In [ ]:
user_restaurant.shape

In [11]:
business_restaurant=business.loc[business['business_id'].isin(review_restaurant['business_id'].unique())]

In [14]:
categories_df = pd.read_csv('../data/all-restaurants/categories.csv')
cuisine = categories_df['cuisine'].tolist()
food = categories_df['food'].tolist()
rest_type = categories_df['rest_type'].tolist()
diet = categories_df['diet'].tolist()

In [15]:
writer = open('../data/restaurants/kg.txt', 'w', encoding='utf-8')
writer2 = open('../data/restaurants/item_index2entity_id.txt', 'w', encoding='utf-8')
entity_cnt = 0
entities = []
for index, row in business_restaurant.iterrows():
    writer2.write('%s\t%d\n' % (row['business_id'], entity_cnt))
    entity_cnt+=1
    entities.append(row['business_id'])
    for c in row['categories'].split(", "):
        if c != 'Restaurants' and c != 'Food':
            valid = True
            if c in cuisine:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'cuisine', c))
            elif c in food:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'food', c))
            elif c in rest_type:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'rest_type', c))
            elif c in diet:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'diet', c))
            else:
                valid = False
            #else:
            #    writer.write('%s\t%s\t%s\n' % (row['business_id'], 'business_type', c))
            if valid and c not in entities:
                writer2.write('%s\t%d\n' % (c, entity_cnt))
                entity_cnt+=1
                entities.append(c)

    writer.write('%s\t%s\t%s\n' % (row['business_id'], 'stars', row['stars']))
    if str(row['stars']) + "_stars" not in entities:
        writer2.write('%s\t%s\n' % (row['stars'], entity_cnt))
        entity_cnt+=1
        entities.append(str(row['stars']) + "_stars")
    if row['state'] not in entities:
        writer2.write('%s\t%s\n' % (row['state'], entity_cnt))
        entity_cnt+=1
        entities.append(row['state'])
    city = row['city'] + "," + row['state']
    writer.write('%s\t%s\t%s\n' % (row['business_id'], 'city', city))
    if city not in entities:
        writer2.write('%s\t%s\n' % (city, entity_cnt))
        entity_cnt+=1
        entities.append(city)
        writer.write('%s\t%s\t%s\n' % (city, 'state', row['state']))
writer.close()
writer2.close()

In [16]:
review_ratings = review_restaurant[['user_id', 'business_id', 'stars']]

In [17]:
review_ratings.head()

,user_id,business_id,stars
0,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0
3,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0
4,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0
5,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,1.0
6,Q8c91v7luItVB0cMFF_mRA,EXOsmAB1s71WePlQk0WZrA,2.0


In [19]:
review_ratings.to_csv('../data/restaurants/ratings.csv', index=False)